In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
import time
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import langdetect
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re


def preprocess_comment(comment):
  # Lowercase the text
  comment = comment.lower()

  # Remove punctuation (optional)
  # You can uncomment this line if you want to remove punctuation
  comment = re.sub(r'[^\w\s]', '', comment)

  # Tokenize the comment
  tokens = nltk.word_tokenize(comment)

  # Remove stop words
  stop_words = stopwords.words('english')
  tokens = [token for token in tokens if token not in stop_words]

  # Apply stemming (optional)
  # You can uncomment this line if you want to use stemming
  ps = PorterStemmer()
  tokens = [ps.stem(token) for token in tokens]

  # Handle emojis (optional)
  # You can implement emoji handling here (e.g., convert to text descriptions)

  # Join the preprocessed tokens back into text
  preprocessed_comment = ' '.join(tokens)

  return preprocessed_comment



def detect_language(text):
  """
  This function detects the language of a text using langdetect.
  """
  try:
    language = langdetect.detect(text)
    return language
  except langdetect.lang_detect_exception.LangDetectException:    
    return "Cannot detect the language"


pageNo = 1

while pageNo < 10:
  website_url = "https://letterboxd.com/film/her/reviews/by/added/page/"+str(pageNo)+"/"
  
  pageNo =pageNo+1
  driver = webdriver.Chrome()

  try:
    driver.get(website_url)
  except Exception as e:    
    pageNo =pageNo+1
    website_url = "https://letterboxd.com/film/her/reviews/by/added/page/"+str(pageNo)+"/"
    driver.get(website_url)      
  
  
  time.sleep(10)
 
 
  # Find the button using XPath that combines button and child p tag with text 'consent'
  consent_button = driver.find_elements(By.CLASS_NAME, "fc-cta-consent")    
  if consent_button:
    # Click the button
    consent_button[0].click()
  time.sleep(1)  


  links_within_div = driver.find_elements(By.XPATH, ".//a")  
  if links_within_div:
    for i in reversed(links_within_div):
      if i.text == "more":
        i.click()

  time.sleep(1)
  # Find all divs with the specified class
  div_elements = driver.find_elements(By.CLASS_NAME, "collapsible-text")
  analyzer = SentimentIntensityAnalyzer()

  extracted_text = []
  # Process each div element
  for div in div_elements:
    # Find all paragraph (p) tags within the div
    paragraph_elements = div.find_elements(By.TAG_NAME, "p")


    # Process each paragraph element (print text)
    for paragraph in paragraph_elements:
      userComment = paragraph.text      
      userComment = userComment.replace("This review may contain spoilers.","")      
      if detect_language(userComment) == "en":   
        userComment = preprocess_comment(userComment)     
        scores = analyzer.polarity_scores(userComment)
        extracted_text.append([userComment +"***"+  str(scores['compound']) + ','])
        print(userComment +"***"+  str(scores['compound']) + ',')             

  with open("data.csv", "w", newline="") as csvfile:
    # Create a CSV writer object with the default comma delimiter
    writer = csv.writer(csvfile)
    # Write the data to the CSV file
    writer.writerows(extracted_text)

  driver.close()



handl truth***0.3182,
nice soundtrack***0.4215,
get theodor also wan na get marri charact ai bot***0.0,
horror movi***-0.5719,
love film must say ten time better watch right lost translat***0.7003,
cant explain hard one hit***0.0762,
learn four thing***0.0,
1 whole ai panic thing that go right also thing back 2013 mean cours ive seen film megalomaniac comput go back 80 ad beyond one point thought 2020***-0.5106,
2 idea fall love interfac lot less ludicr scarlett johansson whisper ear***0.6369,
3 spike jonz direct lot less featur film thought***0.0,
4 joaquin phoenix spit imag colin farrel look***0.0,
add list lone person neon citi***-0.2732,
mix feel understand yall love actual good cant realli get***0.7964,
janet said 2013 ikea threw movi***0.0,
comfort movi***0.3612,
ish depress***-0.4939,
handl truth***0.3182,
okay win im love oper system stuff guess im woke***0.872,
interest movi sad ahead time incred***-0.0258,
love felt like watch realli great black mirror episod beauti cinematog